# Phylogenetics-script-Pseudomonas-Syringae_S.Pflanze

#### Constructing phylogenetic trees for the unknown Pseudomonas

Author: AI 

Date: 01.2023

### 00 - Conda env buildup and versions

In [ ]:
# Conda create env = Blasty
conda create -y -n blasty -c conda-forge -c bioconda blast taxonkit biopython

# Conda create env = phylogenetics
conda create --name phylogenetics -c bioconda  gettools clustalo seqkit

# Create conda env = bit
conda create -n bit -c conda-forge -c bioconda -c defaults -c astrobiomike bit

## 01- Prepare Input Output and Scripts directories 

You can find the github repository of the project [here](https://github.com/Darcy220606/pseudomonas_Spflanze.git)

In [ ]:
git clone https://github.com/Darcy220606/NRPS-Pnunensis4A2e.git

**Project directory**
```
.
├── 01-resources
├── 02-scripts
├── 03-data
├── 04-analysis
├── 05-results
├── 06-publication
└── README.md
```

#### Change into the directory

In [ ]:
cd /NRPS-Pnunensis4A2e

## 02- Pseudomonas refseq - Gtotree (complete and rep. genomes) 
#### Which strain is the closest? 

This is based on [gtotree](https://github.com/AstrobioMike/GToTree/wiki/example-usage#alteromonas-example)

A. Find out where this strain belongs in the Pseudomonas refseq NCBI db by consructing a de novo phylogenomic tree.
   
   1. Download the root sequence manually. We chose [Aestuariirhabdus haliotis](https://www.ncbi.nlm.nih.gov/labs/data-hub/genome/GCF_023509475.1/) located in `04-analysis/gtotree/GCF_023509475.1_ASM2350947v1_genomic.fna`
   
   2. Download accession numbers as below. Located in `04-analysis/gtotree/assembly-accs-complete-genomes.txt` (1432)
   
   3. Create a list with genome_to_id_map by typing in the name of the root and the unknown genome : MANUALLY located in `/04-analysis/gtotree/genome_to_id_map.tsv`

In [ ]:
#!/bin/bash

eval "$(conda shell.bash hook)"

PROJECT_DIR=NRPS-Pnunensis4A2e
REF_GENOME=/Net/Groups/ccdata/databases/ncbi-ref-genomes/pseudomonas/completegenomes #representative
AN_DIR=$PROJECT_DIR/04-analysis/gtotree

mkdir $REF_GENOME
mkdir $AN_DIR

##################################################################
# NCBI: Download refgenomes of Pseudomonas :  27461(total) -to- 25510 (fasta) genomes = memory issues
#                                             14,376 (total refseq) -to- 14377 (fasta) genomes = memory issues
#                                             1432 (total refseq-comp genomes) -to- 1372 (fasta) genomes
#                                             321 (total refseq-rep genomes) -to-  (fasta) genomes
##################################################################
conda activate blasty

cd $REF_GENOME

#OPTION#1: Grab the search query from the Assembly database on ncbi first  :: Pseudomonas-complete genomes-ref/gen
esearch -db assembly -query '"Pseudomonas"[Organism] OR Pseudomonas[All Fields]) AND (("latest refseq"[filter] OR "latest genbank"[filter]) AND "complete genome"[filter] AND all[filter] NOT anomalous[filter]' \
| esummary | xtract -pattern DocumentSummary -element AssemblyAccession > $AN_DIR/assembly-accs-complete-genomes.txt

##################################################################
# Create a list with genome_to_id_map
##################################################################

# Create a file with fasta and gbff locations with 
cd $AN_DIR
ls *.fa > fasta_files.txt
ls *.gbff > genbank_files.txt

##################################################################
# Construct the phylogenetic tree
##################################################################

conda activate gtotree

cd $AN_DIR

GToTree -a $AN_DIR/assembly-accs-complete-genomes.txt -g genbank_files.txt \
        -f fasta_files.txt -H Gammaproteobacteria \
        -t -L Species,Strain -m genome_to_id_map.tsv -j 11 \
        -o pseudomonas_refseq-compgenomes
        
conda deactivate

# RUN results: 
# 1. 1 genome accession removed GCF_000902895.1 - for too few hits
# 2. 1292 of the total 1372 input accessions had their genomes successfully downloaded and searched
# 3. Visualise trees *.tre with itol

##################################################################
# Nucleutide based tree on the clade containing the 4A2E genome
##################################################################
# MANUALLY: prune the tree to the seq. on teh clade that contains the 4A2E genome (19 genomes)
#Found here: `/04-analysis/gtotree/pruned_acc.txt`

##################################################################
# RUN fastANI and plot a tree  (ANI)
##################################################################

## Download the fasta files in the clade containing the genome isolate 
conda activate bit

# Download the fasta files corresponnding to the accessions
# -j parallelism 
ANI=$PROJECT_DIR/04-analysis/gtotree/ANI
ACC=$PROJECT_DIR/04-analysis/gtotree

mkdir $ANI
cd $ANI

bit-dl-ncbi-assemblies -w $ACC/pruned_acc.txt -f fasta -j 30
# Result: ?? genomes downloaded
gzip -d *.gz 

cp $PROJECT_DIR/04-analysis/NG-16087_Pseudomonas_4A2E_2000000_lib257634_5948_paired_assembly.fa $ANI/
ls $ANI > acc_IDs.txt

conda deactivate bit

conda activate fastani
#fragLen 3000
fastANI --ql acc_IDs.txt --rl acc_IDs.txt -t 15 -k 16 --fragLen 3000 -o ANI_output_fg3000.txt --matrix

awk -F '\t' '{print $1"\t"$2"\t"$3}' ANI_output_fg3000.txt > ANI_output1_fg3000.txt
awk 'BEGIN {FS=OFS="\t"} 
           {col[$1]; row[$2]; val[$2,$1]=$3}
     END   {for(c in col) printf "%s", OFS c; print "";
            for(r in row)
              {printf "%s", r;
               for(c in col) printf "%s", OFS val[r,c]
               print ""}}' ANI_output1_fg3000.txt > ANI_OUT_MATRIX_fg3000.txt

# using the matrix as input run the R script ANI.R

##################################################################
# Plot the heat map for ANI 
##################################################################
# Found here `NRPS-Pnunensis4A2e/02-scripts/ANI.R`

## 03- Mycin BGC - BLAST 
#### Where is the BGC also found ? 

In [ ]:

#!/bin/bash
# running with blast4
eval "$(conda shell.bash hook)"

PROJECT_DIR=NRPS-Pnunensis4A2e
AN_DIR=$PROJECT_DIR/04-analysis/blast
NCBI=/Net/Groups/ccdata/databases/ncbi-nt

mkdir $AN_DIR
cd $AN_DIR

##################################################################
# BLAST -remote # Downloaded on 13/04/2023
##################################################################
conda activate blasty

cd $NCBI
# I updated the nt database on our server by running: (Only do once)
# And for future purposes it can be updated by running only 'update_blastdb.pl --passive --force blastdb nt'

wget $NCBI "ftp://ftp.ncbi.nlm.nih.gov/blast/db/nt.*tar.gz"
for f in nt.??.tar.gz; do
tar -xvzf $f;
done

cd $AN_DIR

mkdir $AN_DIR/blastn_corrected
mkdir $AN_DIR/tblastn_corrected

#blastn
blastn -query $PROJECT_DIR/01-resources/tycCmycin_nt.fasta -db $NCBI/nt \
-max_target_seqs 100 \
-num_threads 25 \
-task megablast \
-evalue 0.05 \
-outfmt "6 qseqid sseqid pident sacc mismatch sseq sstart send evalue sscinames nident mismatch length qcovs qcovhs" \
-out matches_nt_blastn_fna_qcov_local_5.txt

python3 /home/aibrahim/jupyter/jupyter-notebooks-uptodate/blastn_filter_hsps.py \
--blastn_file matches_nt_blastn_fna_qcov_local_5.txt \
--output_folder $AN_DIR/blastn_corrected

awk -v FS='[\t]' -v OFS='\t' 'NR > 1 { print $2":"$10"\t"$6 }' $AN_DIR/blastn_corrected/matches_nt_blastn_fna_qcov_local_filtered.txt > $AN_DIR/blastn_corrected/matches_combined_blastn_local.txt
awk -F "\t" '{print ">"$1"\n"$2}'  $AN_DIR/blastn_corrected/matches_combined_blastn_local.txt >  $AN_DIR/blastn_corrected/matches_blastn_local.fa
sed 's/ /_/g' -i $AN_DIR/blastn_corrected/matches_blastn_local.fa
sed 's/[][]//g' -i $AN_DIR/blastn_corrected/matches_blastn_local.fa
sed 's/:/_/g' -i $AN_DIR/blastn_corrected/matches_blastn_local.fa
sed 's/,//g' -i $AN_DIR/blastn_corrected/matches_blastn_local.fa
sed 's/[()]//g' -i $AN_DIR/blastn_corrected/matches_blastn_local.fa
sed "s/'//g" -i $AN_DIR/blastn_corrected/matches_blastn_local.fa
sed "s/'//g" -i $AN_DIR/blastn_corrected/matches_blastn_local.fa
sed "s/;//g" -i $AN_DIR/blastn_corrected/matches_blastn_local.fa

cat $PROJECT_DIR/01-resources/tycCmycin_nt.fasta $AN_DIR/blastn_corrected/matches_blastn_local.fa > $AN_DIR/blastn_corrected/matches_blastn_local.fasta

# tblastn
tblastn -query $PROJECT_DIR/01-resources/mycin_nrps_protein.fasta -db $NCBI/nt \
-max_target_seqs 100 \
-num_threads 25 \
-evalue 0.05 \
-word_size 5 \
-outfmt "6 qseqid sseqid pident sacc mismatch sseq sstart send evalue sscinames nident mismatch length qcovs qcovhsp" \
-out $AN_DIR/tblastn_corrected/matches_nt_tblastn_faa_qcov_local_5.txt

python3 /home/aibrahim/jupyter/jupyter-notebooks-uptodate/blastn_filter_hsps.py \
--blastn_file $AN_DIR/tblastn_corrected/matches_nt_tblastn_faa_qcov_local_5.txt \
--output_folder $AN_DIR/tblastn_corrected/

awk -v FS='[\t]' -v OFS='\t' 'NR > 1 { print $2":"$10"\t"$6 }' $AN_DIR/tblastn_corrected/matches_nt_blastn_fna_qcov_local_filtered.txt > $AN_DIR/tblastn_corrected/matches_combined_tblastn_local.txt
awk -F "\t" '{print ">"$1"\n"$2}'  $AN_DIR/tblastn_corrected/matches_combined_tblastn_local.txt >  $AN_DIR/tblastn_corrected/matches_tblastn_local.fa
sed 's/ /_/g' -i  $AN_DIR/tblastn_corrected/matches_tblastn_local.fa
sed 's/[][]//g' -i  $AN_DIR/tblastn_corrected/matches_tblastn_local.fa
sed 's/:/_/g' -i  $AN_DIR/tblastn_corrected/matches_tblastn_local.fa
sed 's/,//g' -i  $AN_DIR/tblastn_corrected/matches_tblastn_local.fa
sed 's/[()]//g' -i  $AN_DIR/tblastn_corrected/matches_tblastn_local.fa
sed "s/'//g" -i  $AN_DIR/tblastn_corrected/matches_tblastn_local.fa
sed "s/'//g" -i  $AN_DIR/tblastn_corrected/matches_tblastn_local.fa
sed "s/;//g" -i  $AN_DIR/tblastn_corrected/matches_tblastn_local.fa

cat $PROJECT_DIR/01-resources/tycCmycin_aa.fasta $AN_DIR/tblastn_corrected/matches_tblastn_local.fa > $AN_DIR/tblastn_corrected/matches_tblastn_local.fasta

##################################################################
# MUSCLE
##################################################################
# PROBLEM of the mycin tree: The alignments turned out to be sqewed in the resulting ncbi table therfeore, the accessions and the coordinates have to be taken into consideration instead 

conda activate /home/aibrahim/anaconda3/envs/phylogenetics 

muscle -in $AN_DIR/blastn_corrected/matches_blastn_local.fasta -out $AN_DIR/blastn_corrected/matches_blastn_local.afa -maxiters 2
muscle -in $AN_DIR/tblastn_corrected/matches_tblastn_local.fasta -out $AN_DIR/tblastn_corrected/matches_tblastn.afa -maxiters 4

conda deactivate 

conda activate raxml

# raxml hates illegal characters : remove them
cd $AN_DIR/blastn_corrected
raxmlHPC-PTHREADS-SSE3 -T 25 -f a -m GTRGAMMA -p 1989 -x 1989 -N 100 -s matches_blastn_local.afa -n bootstrap_matches_blastn_local.tre
cd $AN_DIR/tblastn_corrected
raxmlHPC-PTHREADS-SSE3 -T 25 -f a -m PROTGAMMAAUTO -p 1989 -x 1989 -N 100 -s matches_tblastn.afa -n bootstrap_matches_tblastn.tre

conda deactivate 

##################################################################
# Add the accession labels to the tree
##################################################################

# grab the accession ids from the blast results 
cd $AN_DIR/blastn_corrected
awk '{print $4}' matches_nt_blastn_fna_qcov_local_filtered.txt > acc_list_blastn_local.txt
cd $AN_DIR/tblastn_corrected
awk '{print $4}' matches_nt_blastn_fna_qcov_local_filtered.txt > acc_list_tblastn.txt

conda activate blasty

# find the taxa label in NCBI
cd $AN_DIR/blastn_corrected
cat acc_list_blastn_local.txt | epost -db nuccore | esummary -db nuccore \
| xtract -pattern DocumentSummary -element Caption,Organism,Description > acc_list_blastn_local_taxid.txt
cd $AN_DIR/tblastn_corrected
cat acc_list_tblastn.txt | epost -db nuccore | esummary -db nuccore \
| xtract -pattern DocumentSummary -element Caption,Organism,Description > acc_list_tblastn_taxid.txt


#### Filter the tblastn results and run them on Clinker 

In [ ]:
#!/bin/bash

eval "$(conda shell.bash hook)"

PROJECT_DIR=NRPS-Pnunensis4A2e
AN_DIR=$PROJECT_DIR/04-analysis/blast
NCBI=/Net/Groups/ccdata/databases/ncbi-nt

mkdir $AN_DIR
cd $AN_DIR

##################################################################
# Filter tblastn according to sequence similarity and qcov
##################################################################
cd $AN_DIR/tblastn_corrected/

#Filter tblastn according to sequence similarity/query coverage
awk -v FS='[\t]' '$13 > 98  && $3 > 75.0' matches_nt_blastn_fna_qcov_local_filtered.txt > matches_nt_blastn_fna_qcov_local_filtered_2.txt
#Grab the accession IDs and the regions 
#Extract the sstart and send coordinates  and the tsrand orientation 
#awk -v FS='[\t]' -v OFS='\t' 'NR > 1 { print $2":"$7"-"$8 }' matches2.txt > matches_coords.txt
awk -v FS='[\t]' -v OFS='\t' 'NR > 1 { print $2":"$7"-"$8":"$1}' matches_nt_blastn_fna_qcov_local_filtered_2.txt > extracted.txt

# Extract the accession numbers
#grab the acc and coordinates only from the file
awk -F '|' '$0=$4 FS $5' extracted.txt > extracted_filt.txt
# replace the | and : with tab
sed -i 's/|:/\t/g' extracted_filt.txt
sed -i 's/:/\t/g' extracted_filt.txt
# replace the - with tab 
sed -i 's/-/\t/g' extracted_filt.txt

##################################################################
# Download the gbk/gff files of the genomes 
##################################################################
mkdir fastas
cd fastas

conda activate blasty

# grab it according to coordinates 
while IFS= read -r line; do
    # Perform a command on each line of the input file 
    id=$(echo -e "${line}" | cut -f 1)
    echo "${id}"
    efetch -id $id -db nuccore -format fasta > $id.fasta
done < ../extracted_filt.txt

##############################################
# PROKKA for gene annotation
##############################################
eval "$(conda shell.bash hook)"
conda activate /home/aibrahim/anaconda3/envs/Prokka

mkdir $AN_DIR/tblastn_corrected/prokka

for F in $AN_DIR/tblastn_corrected/fastas/*.fasta; do 
  N=$(basename $F .fasta) ;
  mkdir $AN_DIR/tblastn_corrected/prokka/$N ;
  prokka --quiet --compliant --prefix $N --locustag $N --force --outdir $AN_DIR/tblastn_corrected/prokka/$N --cpus 28  $F ;
done 

conda deactivate

##############################################
# Slice the gbk file
##############################################
# prepared a script that can slice gbk files by giving the gene_id and the number of surrounding genes
 
conda activate biopython

cd $AN_DIR/tblastn_corrected/prokka/
for F in $AN_DIR/tblastn_corrected/prokka/*/*.gbk; do 
N=$(basename $F .gbk)
cd $AN_DIR/tblastn_corrected/prokka/$N
python3 $PROJECT_DIR/02-scripts/gbk_slicer.py $F $N.sliced3.gb tycC_3 41 42 ; done

conda deactivate

##############################################
# Clinker
##############################################
mkdir $AN_DIR/tblastn_corrected/clinker
cd $AN_DIR/tblastn_corrected/clinker

conda activate clinker

clinker $AN_DIR/tblastn_corrected/prokka/*/*sliced3.gb -p mycin3.v2.html -j 28 -o mycin3.v2.aln
clinker $AN_DIR/tblastn_corrected/prokka/*/*sliced3.gb -p mycin3.html -j 50 -o mycin3.aln
clinker $AN_DIR/tblastn_corrected/prokka/CP007410.1/*.sliced2.gb $AN_DIR/tblastn_corrected/prokka/pnunsis4A2E/*.sliced.gb -p mycin2.html -j 50 -o mycin2.aln
clinker $AN_DIR/tblastn_corrected/prokka/*/*sliced.gb -p mycin4.html -j 50 -o mycin4.aln
